In [1]:
import requests
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn import metrics

In [2]:
# Get static data
df = pd.read_csv("data.csv")
df

,activity,alpha,beta,gamma,stamp,x,y,z
0,Resting,0.000000,0.000000,0.000000,1570374923766,0.000000,0.000000,0.000000
1,Resting,0.088301,-0.220315,-0.094421,1570374923821,-0.002889,0.022908,0.514943
2,Resting,0.858527,0.618979,-0.234303,1570374924290,0.026472,0.005218,0.054518
3,Resting,0.368939,0.128517,-0.374185,1570374924228,-0.000056,-0.016999,0.057604
4,Resting,-0.051582,-0.780718,-0.094421,1570374924083,0.006810,0.012668,0.020030
...,...,...,...,...,...,...,...,...
6907,Jumping,70.237511,-13.301056,15.263779,1570377267084,-0.732208,-4.535389,-9.406340
6908,Jumping,163.337906,13.227617,9.454293,1570377267025,0.342494,-7.706435,-9.395259
6909,Jumping,-80.260078,-12.460013,0.285010,1570377267206,1.312402,-1.951948,-9.349979
6910,Jumping,-5.430055,-1.470512,21.214022,1570377267148,-0.039198,-3.325014,-10.366190


In [3]:
# Define windowing function and parameters
X = []
y = []
class_names = []

skip_count = 0

def windowing(window):
    row = [
        window['alpha'].mean(),
        window['beta'].mean(),
        window['gamma'].mean(),
        window['x'].mean(),
        window['y'].mean(),
        window['z'].mean(),

        window['alpha'].var(),
        window['beta'].var(),
        window['gamma'].var(),
        window['x'].var(),
        window['y'].var(),
        window['z'].var()
        ]
    return row

In [4]:
# Apply windows on data
for i, activity in enumerate(list(set([a.strip() for a in list(np.array(df['activity']))]))):
    class_names.append(activity)
    print(activity)
    for index in np.arange(0, (df['activity'] == activity).shape[0], 20):
        window = df[(df['activity'] == activity)][index:index+20]
        if window.shape[0] < 1:
            skip_count += 1
            continue;

        X.append(windowing(window))
        y.append(i)

Resting
Jumping
Walking


In [5]:
# Split data
X_data = pd.DataFrame(X)
y_data = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

In [6]:
# Train classifier
classifier = SVC(kernel="linear")
classifier.fit(X_train, y_train)

SVC(kernel='linear')

In [7]:
# Feature selection with recursive feature elimination
selector = RFE(classifier, 3, step=1)
selector = selector.fit(X_train, y_train)
print("Selector Support: {}".format(selector.support_))
print("Selector Ranking: {}".format(selector.ranking_))

y_pred = selector.predict(X_test)

print("Accuracy={}".format(accuracy_score(y_test, y_pred)))

C:\Tools\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_features_to_select=3 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


Selector Support: [ True False False False False False  True False False False  True False]
Selector Ranking: [ 1  5  3 10  7  9  1  8  4  6  1  2]
Accuracy=1.0


In [8]:
esti = selector.get_params().get('estimator')

In [9]:
print("* class weights:")
print(esti.class_weight_)
print("* coeffs:")
print(esti.coef_)
print("* intercepts:")
print(esti.intercept_)
print("* vectors")
print(esti.support_vectors_)

* class weights:
[1. 1. 1.]
* coeffs:
[[-3.22770841e-06 -1.17541349e-07 -5.18823452e-07 -6.84415278e-08
   4.65956597e-09 -1.73048589e-07 -4.74037064e-04 -8.48034491e-05
  -1.20827302e-04 -7.22562072e-07 -1.10191787e-05 -1.60970513e-05]
 [-6.43564195e-03 -1.99668111e-03 -5.95438805e-03  2.20247842e-04
   7.57113828e-05  2.99044815e-04 -4.27752235e-02  2.36986654e-03
  -1.98414811e-02 -1.29742838e-04 -1.74178902e-04 -7.96220330e-04]
 [-2.24911237e-02  1.52711573e-02  1.02423494e-02 -3.39849965e-04
  -3.25803672e-03  1.35991144e-03  4.48000912e-04  7.12605647e-04
  -8.02586245e-04 -5.25988425e-03  3.74981370e-02  3.43861400e-02]]
* intercepts:
[ 1.02166413  1.82903272 -5.66144629]
* vectors
[[-6.38737425e-01 -2.96703414e+00 -1.58727036e+00 -2.46422417e-02
   2.31052004e-02 -4.72277880e-02  3.06225073e+00  1.59649185e+02
   5.52193671e+01  1.89425944e-02  5.73916052e-02  2.67057386e-01]
 [-1.63545979e+01 -1.84360476e+00  1.22276344e+01  2.20185647e-01
  -3.76664516e+00  1.74016020e-01  1.

In [11]:
#from sklearn_porter import Porter
#porter = Porter(clf, language='java')
#output = porter.export(embed_data=True)
#print(output)